## RoVit Dataset 
(Pascal VOC for traffic objects)

In [142]:
import sys
import os
import xmltodict
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm

%matplotlib notebook

In [2]:
#!{sys.executable} -m pip install --user --upgrade xmltodict

In [136]:
dataset_path = '/home/zvadaada/rovit-dataset'
image_path = os.path.join(dataset_path, 'JPEGImages')
annotations_path = os.path.join(dataset_path, 'Annotations')
display(image_path)
display(annotations_path)

'/home/zvadaada/rovit-dataset/JPEGImages'

'/home/zvadaada/rovit-dataset/Annotations'

In [4]:
annotations_filenames = os.listdir(annotations_path)

### Create All Annotations Dataframe

In [44]:
def parse_annotation_file(annotations_path, filename):
        
    records = []
    
    annotation_file = os.path.join(annotations_path, filename)

    with open(annotation_file) as fd:
        doc = xmltodict.parse(fd.read(), force_list={'object'})['annotation']
    
    image_filename = doc['filename']
    image_shape_w = doc['size']['width']
    image_shape_h = doc['size']['height']
    image_shape_d = doc['size']['depth']
    
    try:
        traffic_objects = doc['object']
    except:
        print('NOTHING!')
        return []
    
    for traffic_object in traffic_objects:
        object_class = traffic_object['name']
        bbox = (traffic_object['bndbox']['xmin'], traffic_object['bndbox']['ymin'])
        x_min = traffic_object['bndbox']['xmin']
        y_min = traffic_object['bndbox']['ymin']
        x_max = traffic_object['bndbox']['xmax']
        y_max = traffic_object['bndbox']['ymax']
        
        record = [image_filename, image_shape_w, image_shape_h, image_shape_d, 
                  x_min, y_min, x_max, y_max,
                  object_class]
        records.append(record)
        
    return records

#parse_annotation_file(annotations_path, '1277384321Image000006.xml')

In [51]:
objects_records = []
annotations_filenames = os.listdir(annotations_path)
for annotations_filename in tqdm(annotations_filenames):
    #print(annotations_filename)
    objects_records = objects_records + parse_annotation_file(annotations_path, annotations_filename)


 12%|█▏        | 12660/106920 [00:14<02:05, 753.41it/s]


 21%|██        | 22436/106920 [00:28<02:07, 665.19it/s]


 29%|██▊       | 30719/106920 [00:42<02:21, 540.02it/s]


 36%|███▌      | 37985/106920 [00:56<02:24, 477.57it/s]


 42%|████▏     | 44457/106920 [01:10<02:25, 430.31it/s]


...

 92%|█████████▏| 98722/106920 [04:45<00:47, 174.32it/s]


 95%|█████████▍| 101165/106920 [04:59<00:34, 168.23it/s]


 97%|█████████▋| 103482/106920 [05:13<00:21, 163.70it/s]


 99%|█████████▉| 105795/106920 [05:27<00:06, 162.79it/s]


100%|██████████| 106920/106920 [05:34<00:00, 319.35it/s]

In [61]:
df = pd.DataFrame(objects_records, columns=['image_filename', 'image_w', 'image_h', 'image_d',
                                            'x_min', 'y_min', 'x_max', 'y_max', 
                                            'class'])

df['image_filename'] = df['image_filename'].astype(str)
df['image_w'] = df['image_w'].astype(np.float16)
df['image_h'] = df['image_h'].astype(np.float16)
df['image_d'] = df['image_d'].astype(np.float16)
df['x_min'] = df['x_min'].astype(np.float16)
df['y_min'] = df['y_min'].astype(np.float16)
df['x_max'] = df['x_max'].astype(np.float16)
df['y_max'] = df['y_max'].astype(np.float16)
df['class'] = df['class'].astype(str)
df['class'] = df['class'].astype('category')

# one hot encoding for classes
df_dummies = pd.get_dummies(df['class'])
df = pd.concat([df, df_dummies], axis=1)

df.to_csv('rovic-dataset.csv')

In [63]:
print(f'Number of records: {len(df)}')

Number of records: 373047


In [54]:
df.head()

,image_filename,image_w,image_h,image_d,x_min,y_min,x_max,y_max,class,bicycle,bus,car,motorbike,person,trafficlight,trafficsignal
0,bus_internet09_029525.jpg,960,720,3,46,166,632,503,bus,0,1,0,0,0,0,0
1,bus_internet09_029525.jpg,960,720,3,6,331,347,607,car,0,0,1,0,0,0,0
2,bus_internet02_001750.jpg,1280,720,3,756,184,1079,565,bus,0,1,0,0,0,0,0
3,bus_internet02_001750.jpg,1280,720,3,353,299,944,630,car,0,0,1,0,0,0,0
4,bus_internet02_001750.jpg,1280,720,3,355,305,920,645,car,0,0,1,0,0,0,0


## Objects Preprocessing

In [120]:
# CONFIG
NUM_CLASSES = df['class'].nunique()
YOLO_WIDTH = 416
YOLO_HEIGHT = 416
NUM_CELLS = 13
NUM_ANCHORS = 3

anchors = [[0.05524553571428571, 0.045619419642857144],
                        [0.022042410714285716, 0.029296875],
                        [0.13853236607142858, 0.10407366071428571]]

In [133]:
def yolo_dataset_format(dataset, yolo_image_shape, num_classes, num_cells, num_anchors):

    # yolo parameters
#     num_classes = self.config.num_classes()
#     yolo_image_width = self.config.image_width()
#     yolo_image_height = self.config.image_height()
    
    
    yolo_image_width = yolo_image_shape[0]
    yolo_image_height = yolo_image_shape[1]

    image_size = int(yolo_image_width)
    cell_size = int(image_size/num_cells)

    training_data = []
    
    # TODO: SIMPLIFY AND CREATE FUNCS
    for image_filename, boxes in tqdm(dataset.groupby(['image_filename'])):
        
        #print(f'Processing image: {image_filename}')
        
        # --------LOAD-IMAGE----------------
#         img = load_img(dataset_path + '/JPEGImages', image_filename)
#         original_image_shape = img.shape
#         # resize image to yolo wxh
#         resized_img = resize_image(img, yolo_image_width, yolo_image_height)
#         resized_image_shape = resized_img.shape
        #images.append(resized_img)
        # -----------------------------------

        # create zeroed out yolo label
        label = np.zeros(shape=(num_cells, num_cells, num_anchors, 5 + num_classes))
        
        for traffic_object in boxes.itertuples(index=None, name=None):
            
            # TRAFFIC OBJECT STRUTURE
            # traffic_object = ('image_filename', 'image_w', 'image_h', 'image_d', 
            #                   'x_min', 'y_min', 'x_max', 'y_max', 
            #                   'class', <ONE-HOT-ENCODING>)
            image_shape = traffic_object[1:4]
            bbox = traffic_object[4:8]
                        
            # calculate size ratios, img - (w, h, 3)
            width_ratio = yolo_image_width/image_shape[0]
            height_ratio = yolo_image_width/image_shape[1]

            # resize cords
            x_min, x_max = width_ratio * float(bbox[0]), width_ratio * float(bbox[2])
            y_min, y_max = height_ratio * float(bbox[1]), height_ratio * float(bbox[3])
            
#             print(f'{x_min}, {y_min}, {x_max}, {y_max}')
#             plot_img(add_bb_to_img(resized_img, int(x_min), int(y_min), int(x_max), int(y_max)))

            if (x_max < x_min or y_max < y_min):
                raise Exception('Invalid groud truth data, Max < Min')

            # convert to x, y, w, h
            x = (x_min + x_max)/2
            y = (y_min + y_max)/2
            w = x_max - x_min
            h = y_max - y_min

            # make x, y relative to its cell origin
            origin_box_x = int(x / cell_size) * cell_size
            origin_box_y = int(y / cell_size) * cell_size
            cell_x = (x - origin_box_x) / cell_size
            cell_y = (y - origin_box_y) / cell_size
        
            # cell index
            c_x, c_y = int(origin_box_x / cell_size), int(origin_box_y / cell_size)

            # class data
            one_hot = traffic_object[9:]

            for i, (rel_anchor_width, rel_anchor_height) in enumerate(anchors):

                # calculate w,h in respect to anchors size
                a_w = w / (rel_anchor_width * image_size)
                a_h = h / (rel_anchor_height * image_size)

                label[c_x, c_y, i, :] = np.concatenate((cell_x, cell_y, a_w, a_h, 1, one_hot), axis=None)

                #print(f'{cell_x}, {cell_y}, {a_w}, {a_h}')
                
                # ---------TESTING---------
#                 x_min = int((((cell_x*cell_size) + c_x*cell_size) - (a_w * (rel_anchor_width * image_size)/2)))
#                 y_min = int((((cell_y*cell_size) + c_y*cell_size) - (a_h * (rel_anchor_height * image_size)/2)))
#                 x_max = int((((cell_x*cell_size) + c_x*cell_size) + (a_w * (rel_anchor_width * image_size)/2)))
#                 y_max = int((((cell_y*cell_size) + c_y*cell_size) + (a_h * (rel_anchor_height * image_size)/2)))
#                 print(f'{x_min}, {y_min}, {x_max}, {y_max}')
                #plot_img(add_bb_to_img(resized_img, x_min, y_min, x_max, y_max))
                # -------------------------

        training_data.append([image_filename, label])
        
    preprocessed_df = pd.DataFrame(training_data, columns=['image_filename', 'label'])

    return preprocessed_df

In [134]:
preprocessed_df = yolo_dataset_format(df, (YOLO_WIDTH, YOLO_HEIGHT), NUM_CLASSES, NUM_CELLS, NUM_ANCHORS)
preprocessed_df.head()










  0%|          | 0/106920 [00:00<?, ?it/s]








  0%|          | 57/106920 [00:00<03:07, 569.78it/s]








  0%|          | 137/106920 [00:00<02:51, 623.50it/s]








  0%|          | 217/106920 [00:00<02:40, 666.35it/s]








  0%|          | 298/106920 [00:00<02:31, 702.03it/s]








  0%|          | 382/106920 [00:00<02:24, 737.83it/s]








  0%|          | 463/106920 [00:00<02:20, 757.92it/s]








  1%|          | 544/106920 [00:00<02:17, 772.68it/s]








  1%|          | 625/106920 [00:00<02:15, 782.17it/s]








  1%|          | 706/106920 [00:00<02:14, 788.98it/s]








  1%|          | 787/106920 [00:01<02:13, 793.51it/s]








  1%|          | 868/106920 [00:01<02:12, 798.14it/s]








  1%|          | 950/106920 [00:01<02:12, 802.02it/s]








  1%|          | 1032/106920 [00:01<02:11, 804.57it/s]








  1%|          | 1114/106920 [00:01<02:11, 806.76it/s]








  1%|          | 1195/106920 [00:01<02:11, 802.85it/s]








  1%|   

  7%|▋         | 7166/106920 [00:09<02:27, 677.53it/s]








  7%|▋         | 7235/106920 [00:09<02:31, 657.71it/s]








  7%|▋         | 7302/106920 [00:09<02:33, 647.28it/s]








  7%|▋         | 7368/106920 [00:09<02:35, 639.50it/s]








  7%|▋         | 7433/106920 [00:09<02:37, 632.15it/s]








  7%|▋         | 7497/106920 [00:09<02:38, 626.61it/s]








  7%|▋         | 7560/106920 [00:09<02:38, 626.09it/s]








  7%|▋         | 7623/106920 [00:10<02:39, 622.46it/s]








  7%|▋         | 7686/106920 [00:10<02:39, 620.95it/s]








  7%|▋         | 7749/106920 [00:10<02:39, 622.65it/s]








  7%|▋         | 7812/106920 [00:10<02:39, 623.09it/s]








  7%|▋         | 7875/106920 [00:10<02:39, 622.48it/s]








  7%|▋         | 7938/106920 [00:10<02:39, 620.78it/s]








  7%|▋         | 8001/106920 [00:10<02:40, 614.60it/s]








  8%|▊         | 8063/106920 [00:10<02:41, 612.92it/s]








  8%|▊         | 8125/106920 [00:10<02:41, 612.75it/s]


 12%|█▏        | 12416/106920 [00:18<02:40, 590.65it/s]








 12%|█▏        | 12478/106920 [00:18<02:38, 597.50it/s]








 12%|█▏        | 12540/106920 [00:18<02:36, 603.29it/s]








 12%|█▏        | 12602/106920 [00:18<02:35, 605.69it/s]








 12%|█▏        | 12663/106920 [00:19<02:35, 606.45it/s]








 12%|█▏        | 12725/106920 [00:19<02:34, 608.30it/s]








 12%|█▏        | 12786/106920 [00:19<02:36, 603.20it/s]








 12%|█▏        | 12847/106920 [00:19<02:38, 593.69it/s]








 12%|█▏        | 12907/106920 [00:19<02:42, 579.24it/s]








 12%|█▏        | 12966/106920 [00:19<02:42, 576.59it/s]








 12%|█▏        | 13026/106920 [00:19<02:41, 582.75it/s]








 12%|█▏        | 13088/106920 [00:19<02:38, 590.56it/s]








 12%|█▏        | 13148/106920 [00:19<02:40, 585.87it/s]








 12%|█▏        | 13208/106920 [00:19<02:39, 587.78it/s]








 12%|█▏        | 13268/106920 [00:20<02:38, 589.94it/s]








 12%|█▏        | 13328/106920 [00:20<02:

 17%|█▋        | 17655/106920 [00:27<02:41, 551.16it/s]








 17%|█▋        | 17711/106920 [00:27<02:44, 543.03it/s]








 17%|█▋        | 17766/106920 [00:28<02:48, 528.87it/s]








 17%|█▋        | 17820/106920 [00:28<02:47, 531.11it/s]








 17%|█▋        | 17876/106920 [00:28<02:45, 536.61it/s]








 17%|█▋        | 17930/106920 [00:28<02:50, 522.42it/s]








 17%|█▋        | 17983/106920 [00:28<02:49, 524.04it/s]








 17%|█▋        | 18036/106920 [00:28<02:51, 518.56it/s]








 17%|█▋        | 18091/106920 [00:28<02:49, 525.10it/s]








 17%|█▋        | 18145/106920 [00:28<02:48, 527.43it/s]








 17%|█▋        | 18201/106920 [00:28<02:46, 533.73it/s]








 17%|█▋        | 18255/106920 [00:29<02:49, 523.82it/s]








 17%|█▋        | 18308/106920 [00:29<02:53, 511.44it/s]








 17%|█▋        | 18363/106920 [00:29<02:50, 519.92it/s]








 17%|█▋        | 18417/106920 [00:29<02:48, 524.35it/s]








 17%|█▋        | 18472/106920 [00:29<02:

 21%|██▏       | 22977/106920 [00:37<02:19, 601.73it/s]








 22%|██▏       | 23038/106920 [00:37<02:19, 602.67it/s]








 22%|██▏       | 23099/106920 [00:37<02:18, 603.85it/s]








 22%|██▏       | 23160/106920 [00:37<02:19, 601.22it/s]








 22%|██▏       | 23221/106920 [00:37<02:19, 599.51it/s]








 22%|██▏       | 23282/106920 [00:37<02:18, 601.72it/s]








 22%|██▏       | 23343/106920 [00:37<02:19, 600.29it/s]








 22%|██▏       | 23404/106920 [00:37<02:18, 602.38it/s]








 22%|██▏       | 23465/106920 [00:37<02:18, 601.32it/s]








 22%|██▏       | 23526/106920 [00:38<02:18, 601.85it/s]








 22%|██▏       | 23587/106920 [00:38<02:18, 600.99it/s]








 22%|██▏       | 23648/106920 [00:38<02:18, 601.18it/s]








 22%|██▏       | 23709/106920 [00:38<02:18, 602.08it/s]








 22%|██▏       | 23770/106920 [00:38<02:18, 600.45it/s]








 22%|██▏       | 23831/106920 [00:38<02:18, 599.57it/s]








 22%|██▏       | 23893/106920 [00:38<02:

...

 77%|███████▋  | 82494/106920 [02:09<00:41, 593.72it/s]








 77%|███████▋  | 82554/106920 [02:09<00:41, 590.26it/s]








 77%|███████▋  | 82615/106920 [02:09<00:40, 595.81it/s]








 77%|███████▋  | 82676/106920 [02:09<00:40, 599.33it/s]








 77%|███████▋  | 82736/106920 [02:09<00:40, 599.44it/s]








 77%|███████▋  | 82799/106920 [02:09<00:39, 606.20it/s]








 77%|███████▋  | 82860/106920 [02:09<00:39, 603.17it/s]








 78%|███████▊  | 82921/106920 [02:09<00:39, 603.17it/s]








 78%|███████▊  | 82982/106920 [02:09<00:39, 602.09it/s]








 78%|███████▊  | 83044/106920 [02:09<00:39, 606.65it/s]








 78%|███████▊  | 83106/106920 [02:10<00:39, 609.70it/s]








 78%|███████▊  | 83168/106920 [02:10<00:38, 611.90it/s]








 78%|███████▊  | 83230/106920 [02:10<00:38, 611.37it/s]








 78%|███████▊  | 83293/106920 [02:10<00:38, 614.07it/s]








 78%|███████▊  | 83355/106920 [02:10<00:38, 614.50it/s]








 78%|███████▊  | 83418/106920 [02:10<00:

 82%|████████▏ | 87955/106920 [02:18<00:31, 602.42it/s]








 82%|████████▏ | 88016/106920 [02:18<00:31, 599.36it/s]








 82%|████████▏ | 88077/106920 [02:18<00:31, 600.60it/s]








 82%|████████▏ | 88138/106920 [02:18<00:31, 597.66it/s]








 82%|████████▏ | 88198/106920 [02:18<00:31, 587.99it/s]








 83%|████████▎ | 88257/106920 [02:18<00:32, 581.35it/s]








 83%|████████▎ | 88316/106920 [02:18<00:32, 579.10it/s]








 83%|████████▎ | 88374/106920 [02:18<00:32, 577.46it/s]








 83%|████████▎ | 88434/106920 [02:19<00:31, 581.39it/s]








 83%|████████▎ | 88497/106920 [02:19<00:31, 593.25it/s]








 83%|████████▎ | 88558/106920 [02:19<00:30, 597.64it/s]








 83%|████████▎ | 88620/106920 [02:19<00:30, 602.66it/s]








 83%|████████▎ | 88683/106920 [02:19<00:30, 607.88it/s]








 83%|████████▎ | 88746/106920 [02:19<00:29, 611.78it/s]








 83%|████████▎ | 88808/106920 [02:19<00:29, 613.99it/s]








 83%|████████▎ | 88870/106920 [02:19<00:

 88%|████████▊ | 93653/106920 [02:27<00:17, 778.37it/s]








 88%|████████▊ | 93732/106920 [02:27<00:16, 776.24it/s]








 88%|████████▊ | 93811/106920 [02:27<00:16, 777.83it/s]








 88%|████████▊ | 93889/106920 [02:27<00:16, 772.22it/s]








 88%|████████▊ | 93969/106920 [02:27<00:16, 779.88it/s]








 88%|████████▊ | 94050/106920 [02:27<00:16, 786.25it/s]








 88%|████████▊ | 94131/106920 [02:27<00:16, 791.77it/s]








 88%|████████▊ | 94211/106920 [02:28<00:16, 790.23it/s]








 88%|████████▊ | 94294/106920 [02:28<00:15, 798.80it/s]








 88%|████████▊ | 94375/106920 [02:28<00:15, 801.70it/s]








 88%|████████▊ | 94456/106920 [02:28<00:15, 801.17it/s]








 88%|████████▊ | 94537/106920 [02:28<00:15, 780.82it/s]








 88%|████████▊ | 94617/106920 [02:28<00:15, 786.34it/s]








 89%|████████▊ | 94696/106920 [02:29<00:32, 376.09it/s]








 89%|████████▊ | 94779/106920 [02:29<00:27, 449.55it/s]








 89%|████████▊ | 94860/106920 [02:29<00:

 94%|█████████▍| 100476/106920 [02:36<00:09, 684.32it/s]








 94%|█████████▍| 100545/106920 [02:37<00:09, 673.76it/s]








 94%|█████████▍| 100616/106920 [02:37<00:09, 684.21it/s]








 94%|█████████▍| 100688/106920 [02:37<00:08, 693.11it/s]








 94%|█████████▍| 100758/106920 [02:37<00:08, 685.12it/s]








 94%|█████████▍| 100828/106920 [02:37<00:08, 689.42it/s]








 94%|█████████▍| 100898/106920 [02:37<00:08, 683.73it/s]








 94%|█████████▍| 100967/106920 [02:37<00:08, 679.53it/s]








 94%|█████████▍| 101038/106920 [02:37<00:08, 686.80it/s]








 95%|█████████▍| 101114/106920 [02:37<00:08, 706.05it/s]








 95%|█████████▍| 101188/106920 [02:37<00:08, 714.56it/s]








 95%|█████████▍| 101264/106920 [02:38<00:07, 725.09it/s]








 95%|█████████▍| 101340/106920 [02:38<00:07, 734.67it/s]








 95%|█████████▍| 101416/106920 [02:38<00:07, 740.36it/s]








 95%|█████████▍| 101491/106920 [02:38<00:07, 734.28it/s]








 95%|█████████▍| 101568/1

,image_filename,label
0,1277104195Image000013.jpg,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. ..."
1,1277104195Image000014.jpg,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. ..."
2,1277104195Image000015.jpg,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. ..."
3,1277104195Image000016.jpg,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. ..."
4,1277104195Image000017.jpg,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. ..."


In [146]:
def generate_tfrecords(df, type='train'):

    image_filenames = np.array(df['image_filename'].values.tolist())
    labels = np.array(df['label'].values.tolist(), dtype=np.float32)

    with tf.python_io.TFRecordWriter(f'{dataset_path}/rovit_{type}.tfrecords') as writer:

        for image_filename, labels in tqdm(zip(image_filenames, labels)):
            
#             img_path = os.path.join(image_path, image_filename)
#             img_raw = tf.read_file(img_path)
#             img = tf.image.decode_image(img_raw)
#             img = tf.image.resize_images(img_tensor, [YOLO_WIDTH, YOLO_HEIGHT])

            example = tf.train.Example(features=tf.train.Features(
                feature={
                    'image_filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_filename.tostring()])),
                    'labels': tf.train.Feature(bytes_list=tf.train.BytesList(value=[labels.tostring()]))
                }))

            writer.write(example.SerializeToString())

In [147]:
generate_tfrecords(preprocessed_df)










0it [00:00, ?it/s]








316it [00:00, 3158.32it/s]








632it [00:00, 3157.83it/s]








956it [00:00, 3180.78it/s]








1286it [00:00, 3214.17it/s]








1614it [00:00, 3232.17it/s]








1931it [00:00, 3212.40it/s]








2248it [00:00, 3199.45it/s]








2565it [00:00, 3189.02it/s]








2883it [00:00, 3184.32it/s]








3201it [00:01, 3182.70it/s]








3521it [00:01, 3185.79it/s]








3839it [00:01, 3182.97it/s]








4165it [00:01, 3204.42it/s]








4496it [00:01, 3232.73it/s]








4818it [00:01, 3221.55it/s]








5139it [00:01, 3202.45it/s]








5459it [00:01, 2917.53it/s]








5755it [00:01, 2750.43it/s]








6035it [00:01, 2650.87it/s]








6305it [00:02, 2583.79it/s]








6567it [00:02, 2533.23it/s]








6823it [00:02, 2492.97it/s]








7075it [00:02, 2466.08it/s]








7324it [00:02, 2443.73it/s]








7570it [00:02, 2430.97it/s]








7814it [00:02, 2422.92it/s]








8060it [00:02, 2432.44it/s]





39014it [00:13, 3219.99it/s]








39344it [00:13, 3242.87it/s]








39674it [00:13, 3257.29it/s]








40001it [00:13, 3260.77it/s]








40328it [00:13, 3232.21it/s]








40652it [00:13, 3225.42it/s]








40975it [00:13, 3224.11it/s]








41301it [00:13, 3231.57it/s]








41625it [00:13, 3200.02it/s]








41946it [00:14, 3179.89it/s]








42265it [00:14, 3178.17it/s]








42590it [00:14, 3198.08it/s]








42911it [00:14, 3200.33it/s]








43232it [00:14, 3190.33it/s]








43553it [00:14, 3194.07it/s]








43874it [00:14, 3198.43it/s]








44195it [00:14, 3201.42it/s]








44520it [00:14, 3213.23it/s]








44842it [00:14, 3211.76it/s]








45166it [00:15, 3218.69it/s]








45488it [00:15, 3190.42it/s]








45808it [00:15, 3173.26it/s]








46126it [00:15, 3169.79it/s]








46444it [00:15, 3165.48it/s]








46764it [00:15, 3172.58it/s]








47082it [00:15, 3172.67it/s]








47407it [00:15, 3192.85it/s]








4

80278it [00:26, 3213.99it/s]








80600it [00:26, 3207.82it/s]








80921it [00:26, 3201.94it/s]








81242it [00:26, 3202.19it/s]








81565it [00:26, 3209.15it/s]








81889it [00:26, 3216.59it/s]








82213it [00:26, 3221.16it/s]








82539it [00:26, 3231.73it/s]








82863it [00:26, 3230.26it/s]








83190it [00:26, 3240.40it/s]








83515it [00:27, 3206.87it/s]








83836it [00:27, 3196.51it/s]








84156it [00:27, 3189.24it/s]








84477it [00:27, 3192.76it/s]








84797it [00:27, 3186.98it/s]








85116it [00:27, 3187.55it/s]








85437it [00:27, 3193.31it/s]








85757it [00:27, 3194.05it/s]








86082it [00:27, 3208.43it/s]








86404it [00:27, 3210.00it/s]








86728it [00:28, 3217.34it/s]








87050it [00:28, 3216.60it/s]








87374it [00:28, 3222.72it/s]








87697it [00:28, 3217.87it/s]








88019it [00:28, 3182.35it/s]








88338it [00:28, 3177.04it/s]








88656it [00:28, 3156.47it/s]








8

In [52]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def load_img(dataset_path, image_path):
    return cv2.imread(os.path.join(dataset_path, image_path))


def add_bb_to_img(img, x_min, y_min, x_max, y_max, is_occulded=False):

    color = (255, 0, 0)  # blue is not occulded
    if is_occulded:
        color = (0, 0, 255)  # red is occulded

    pt1 = (x_min, y_min)
    pt2 = (x_max, y_max)

    return cv2.rectangle(img, pt1, pt2, color, 1)


def plot_img(img):
    plt.imshow(cv2.cvtColor(img, 4))
    plt.show()


def resize_image(img, new_width, new_height):
    return cv2.resize(img, (new_width, new_height, ), interpolation=cv2.INTER_CUBIC)